In [1]:
import rasterio as rio
import rioxarray as rxr
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import MultiPolygon
from rasterio.mask import mask
import numpy as np
import os
thomas_fire_bs = r'D:/imagery/masters/output/S2_tf_dNBR.tif'
thomas_fire_shp = r'D:/imagery/masters/baer/thm2017_Basin_DFPredictions_15min_12mmh.shp'

output_directory = r'D:/imagery/masters/output/'


In [2]:
shape = gpd.read_file(thomas_fire_shp)
#print(shape)
#shape
print(shape.columns.tolist())


['BASIN_ID', 'Shape_Leng', 'Shape_Area', 'Segment_ID', 'Acc_Cl', 'Perim_Buff', 'InsidePeri', 'OutsidePer', 'Fire_ID', 'Fire_Segme', 'L_X1', 'L_X2', 'L_X3', 'V_X1', 'V_X2', 'Shape_Le_1', 'R', 'V_X3', 'X', 'ExpX', 'P', 'PCl', 'PCl_Legend', 'LnV', 'Volume', 'VolMin', 'VolMax', 'VolCl', 'VolCl_Lege', 'CombHaz', 'CombHazCl', 'CombHazCl_', 'geometry']


In [3]:
with rio.open(thomas_fire_bs) as src:
    for idx, row in shape.iterrows():
        basin_id = row['BASIN_ID']
        geom = [row['geometry']]

        #Grab individual basin X values.
        X1 = row['L_X1']
        X2 = row['L_X2']
        X3 = row['L_X3']
        
        out_image, out_transform = mask(src, geom, crop=True)
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform,
            "crs": src.crs,
            "nodata": np.nan
        })

        out_image = np.where(out_image == 0, np.nan, out_image)

        out_tif_path = os.path.join(output_directory + f'basin_{basin_id}_dNBR.tif')
        with rio.open(out_tif_path, "w", **out_meta) as dest:
            dest.write(out_image)
            dest.update_tags(X1=str(X1), X2=str(X2), X3=str(X3))
        